In [1]:
import torch
import torch.nn as nn

In [2]:
!pip3 install -U ncps  datasets #pytorch-lightning no hace falta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.5 MB/s eta 0:00:00


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
# Cargar el modelo GPT-2
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from ncps.torch import LTC  # Importamos LTC desde el repositorio

class ModifiedGPT2MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ModifiedGPT2MLP, self).__init__()
        # Reemplazar la capa convencional por la capa líquida LTC
        self.ltc_layer = LTC(input_dim, hidden_dim)  # Usamos LTC en lugar de LiquidLayer
        #self.linear = nn.Linear(hidden_dim, output_dim)  # Capa lineal de salida

    def forward(self, x):
        # Pasar por la capa líquida LTC
        x = self.ltc_layer(x)
        # Luego pasarlo por la capa lineal
        #x = self.linear(x)
        return x

# Reemplazar GPT2MLP con el nuevo MLP modificado en todo el modelo
for block in model.transformer.h:
    block.mlp = ModifiedGPT2MLP(768, 768, 768)

In [ ]:
print(model)

In [ ]:
# Congelar todas las capas excepto las nuevas capas líquidas
for param in model.parameters():
    param.requires_grad = False

# Asegurarse de que las nuevas capas líquidas sean entrenables
for param in model.transformer.h[0].mlp.ltc_layer.parameters():
    param.requires_grad = True


In [ ]:
# Verifica si hay GPU disponible, si no, usa la CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Entrenando en: {device}")

In [ ]:
from datasets import load_dataset
from transformers import GPT2Tokenizer

# Cargar el dataset de ejemplo, he cogido el que se usa en mazo tutoriales
dataset = load_dataset("yelp_review_full")
#Vemos el tamaño del dataset y sus claves
print(dataset)
#Sus keys
print(dataset.keys())
#Reducimos el tamaño del dataset
dataset['train'] = dataset['train'].select(range(100))
dataset['test'] = dataset['test'].select(range(100))
# Agregar un token de padding (esto es necesario para el padding en secuencias de diferentes longitudes)
tokenizer.pad_token = tokenizer.eos_token  # Usar el token EOS como token de padding

# Función para tokenizar los datos
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Tokenizar los datos de entrenamiento y prueba
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Preparamos los datos para el DataLoader
from torch.utils.data import DataLoader

train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]


train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2)

In [ ]:

# Obtener un batch de datos del DataLoader de entrenamiento
for batch in train_dataloader:
    # Imprimir las claves del diccionario del batch
    print(batch.keys())
    # Imprimir los elementos del diccionario del batch
    for key, value in batch.items():
        print(f"Clave: {key}, Valor (tipo): {type(value)}, Forma: {value.shape if isinstance(value, torch.Tensor) else 'N/A'}, Primeros 5 elementos: {value[:5] if isinstance(value, torch.Tensor) else value[:5]}")
    break # Salir del bucle después del primer batch


In [ ]:


import sys

# Obtener el tamaño del batch actual en memoria
def get_batch_memory_usage(batch):
    total_memory = 0
    for key, value in batch.items():
        if isinstance(value, torch.Tensor):
            total_memory += value.element_size() * value.nelement()
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, torch.Tensor):
                    total_memory += item.element_size() * item.nelement()

    return total_memory


# Obtener un batch de datos del DataLoader de entrenamiento
for batch in train_dataloader:
    # Imprimir las claves del diccionario del batch
    print(batch.keys())
    # Imprimir los elementos del diccionario del batch
    for key, value in batch.items():
        print(f"Clave: {key}, Valor (tipo): {type(value)}, Forma: {value.shape if isinstance(value, torch.Tensor) else 'N/A'}, Primeros 5 elementos: {value[:5] if isinstance(value, torch.Tensor) else value[:5]}")

    # Calcula y muestra el tamaño del batch en memoria
    batch_memory = get_batch_memory_usage(batch)
    print(f"Tamaño del batch en memoria: {batch_memory / (1024**2):.2f} MB")  # En megabytes
    break # Salir del bucle después del primer batch


In [ ]:

# Calculate the total number of parameters in the model
total_params = sum(p.numel() for p in model.parameters())

# Calculate the size of the model in MB
model_size_mb = total_params * 4 / (1024**2)  # Assuming 4 bytes per parameter (float32)

print(f"Total number of parameters: {total_params}")
print(f"Model size: {model_size_mb:.2f} MB")


In [ ]:
#Liberamos la memoria de la gpu
torch.cuda.empty_cache()

In [ ]:
from transformers import AdamW
import torch

# Mover el modelo a la GPU
model = model.to(device)

# Definir el optimizador (usamos AdamW para los modelos de Hugging Face)
optimizer = AdamW(model.parameters(), lr=5e-5)

# 1. Definir la función de pérdida (en este caso, CrossEntropyLoss)
criterion = torch.nn.CrossEntropyLoss()

from torch.cuda.amp import autocast, GradScaler

# 2. Inicializar scaler para mixed precision
scaler = GradScaler()

# 3. Configurar acumulación de gradientes
accumulation_steps = 2

# Ciclo de entrenamiento
model.train()
for epoch in range(3):
    running_loss = 0.0
    optimizer.zero_grad()  # Limpiar gradientes al inicio de la época

    for i, batch in enumerate(train_dataloader):
        # Mover datos a GPU
        inputs = torch.stack(batch['input_ids']).to(device)
        attention_mask = torch.stack(batch['attention_mask']).to(device)
        labels = batch['label'].to(device)

        # Forward pass con mixed precision
        with autocast():
            outputs = model(inputs, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss / accumulation_steps  # Normalizar pérdida

        # Backward pass con scaling
        scaler.scale(loss).backward()

        # Actualizar pesos después de acumular gradientes
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_dataloader):
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        # Registrar pérdida y liberar memoria
        running_loss += loss.item() * accumulation_steps
        del outputs, loss
        torch.cuda.empty_cache()

    avg_loss = running_loss / len(train_dataloader)
    print(f'Epoca [{epoch+1}/3], Pérdida Promedio: {avg_loss:.4f}')